In [1]:
import pandas as pd
import sys

sys.path.append("..")
from structure import scriptures_structure

alma_32_files = scriptures_structure.get_embedding_file_path('alma/32/40/20') 
alma_32_embedds = pd.read_csv("../" + alma_32_files[0], header=None).to_numpy()
alma_32_offsets = pd.read_csv("../" + alma_32_files[1], header=None).to_numpy()

# isa_11_files = scriptures_structure.get_embedding_file_path('isa/11/100') 
# isa_11_100_embedds = pd.read_csv("." + isa_11_files[0], header=None).to_numpy()
# isa_11_100_offsets = pd.read_csv("." + isa_11_files[1], header=None).to_numpy()

# ne_30_files = scriptures_structure.get_embedding_file_path('2-ne/30/50')
# print(ne_30_files)
# ne_30_50_embedds = pd.read_csv("." + ne_30_files[0], header=None).to_numpy()
# ne_30_50_offsets = pd.read_csv("." + ne_30_files[1], header=None).to_numpy()

# isa_11_files = scriptures_structure.get_embedding_file_path('isa/11/50') 
# isa_11_50_embedds = pd.read_csv("." + isa_11_files[0], header=None).to_numpy()
# isa_11_50_offsets = pd.read_csv("." + isa_11_files[1], header=None).to_numpy()

    

In [2]:
def get_texts_from_offset(offsets, book_chapter):
    text_array = []
    text_file = "../" + scriptures_structure.get_text_file_path(book_chapter)
    with open(text_file, 'r', encoding='utf-8') as file:
        contents = file.read()
        for offset in offsets:
            # print(contents[offset[0]:offset[1]])
            # print("-----------------------------------")
            text_array.append(contents[offset[0]:offset[1]])
    return text_array

In [3]:
file = "../" + scriptures_structure.get_text_file_path("alma/32")
with open(file, 'r', encoding='utf-8') as f:
    f.seek(434)
    print(f.read(668-434))
    
    

were not permitted to enter into their synagogues to worship God, being esteemed as filthiness; therefore they were poor; yea, they were esteemed by their brethren as dross; therefore they were poor as to things of the world; and also


In [4]:
def convert_text(text):
    lines = []
    text = text.split()
    for i in range(0, len(text), 10):
        line = ' '.join(text[i:i + 10])
        lines.append(line)

    result = '<br>'.join(lines)
    # print(result)
    return result

text = get_texts_from_offset(alma_32_offsets, 'alma/32')
for i in range(len(text)):
    # print("----------------------------------------------------------------------")
    # print(text[i])
    text[i] = convert_text(text[i])

In [5]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.mixture import GaussianMixture
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# def convert_text(text):
#     lines = []
#     text = text.split()
#     for i in range(0, len(text), 10):
#         line = ' '.join(text[i:i + 10])
#         lines.append(line)

#     result = '<br>'.join(lines)
#     # print(result)
#     return result

def convert_all_text(text):
    for i in range(len(text)):
        text[i] = convert_text(text[i])
    return text

text = get_texts_from_offset(alma_32_offsets, 'alma/32')
print(text)

pca = PCA(n_components=2)
reduced_embeddings = pca.fit_transform(alma_32_embedds)

kmeans = KMeans(n_clusters=3, random_state=0).fit(reduced_embeddings)
cluster_labels = kmeans.labels_

# pca = PCA(n_components=2)
# reduced_embeddings = pca.fit_transform(alma_32_embedds)

# plt.scatter(reduced_embeddings[:, 0], reduced_embeddings[:, 1], c=cluster_labels)
# plt.title('K-Means Clustering of Text Embeddings')
# plt.show()

data = {'x': reduced_embeddings[:, 0],
        'y': reduced_embeddings[:, 1],
        'text_labels': convert_all_text(text),
        'cluster_labels': cluster_labels}
df = pd.DataFrame(data)


fig = go.Figure(go.Scatter(
    x=df['x'],
    y=df['y'],
    hovertext=df['text_labels'],
    mode='markers',
    marker=dict(color=cluster_labels)
))

# Customize the hovertemplate to show hovertext
fig.update_traces(marker=dict(size=10), hovertemplate='%{hovertext}')
fig.update_layout(title='KMeans Model Clustering of Alma 32')
fig.show()

gmm = GaussianMixture(n_components=3)
gmm.fit(reduced_embeddings)
cluster_labels = gmm.predict(reduced_embeddings)

fig = go.Figure(go.Scatter(
    x=df['x'],
    y=df['y'],
    hovertext=df['text_labels'],
    mode='markers',
    marker=dict(color=cluster_labels)
))

for cluster in set(cluster_labels):
    filtered_df = df[df['cluster_labels'] == cluster]
    text_for_cluster = filtered_df['text_labels'].tolist()
    print("Cluster " + str(cluster) + ":")
    for text in text_for_cluster:
        print(' '.join(text.split("<br>")))
        print("\n")

fig.update_traces(marker=dict(size=10), hovertemplate='%{hovertext}')
fig.update_layout(title='Gaussian Mixture Model Clustering of Alma 32')
fig.show()

['And it came to pass that they did go forth, and began to preach the word of God unto the people, entering into their synagogues, and into their houses; yea, and even they did preach the word in their streets.', 'people, entering into their synagogues, and into their houses; yea, and even they did preach the word in their streets.\nAnd it came to pass that after much labor among them, they began to have success among the poor class', 'And it came to pass that after much labor among them, they began to have success among the poor class of people; for behold, they were cast out of the synagogues because of the coarseness of their apparel—\nTherefore they w', 'of people; for behold, they were cast out of the synagogues because of the coarseness of their apparel—\nTherefore they were not permitted to enter into their synagogues to worship God, being esteemed as filthiness; therefore they were poor; yea, t', 're not permitted to enter into their synagogues to worship God, being esteemed as

C:\Users\rylee\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Cluster 0:
And it came to pass that they did go forth, and began to preach the word of God unto the people, entering into their synagogues, and into their houses; yea, and even they did preach the word in their streets.


people, entering into their synagogues, and into their houses; yea, and even they did preach the word in their streets. And it came to pass that after much labor among them, they began to have success among the poor class


And it came to pass that after much labor among them, they began to have success among the poor class of people; for behold, they were cast out of the synagogues because of the coarseness of their apparel— Therefore they w


of people; for behold, they were cast out of the synagogues because of the coarseness of their apparel— Therefore they were not permitted to enter into their synagogues to worship God, being esteemed as filthiness; therefore they were poor; yea, t


re not permitted to enter into their synagogues to worship God, being esteemed 

In [6]:
import plotly.express as px
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import numpy as np

# Perform PCA to reduce dimensionality for visualization
pca = PCA(n_components=2)
ne_30_100_embedds_2d = pca.fit_transform(ne_30_100_embedds)
text = get_texts_from_offsets(ne_30_100_offsets)
# Perform K-Means clustering
kmeans = KMeans(n_clusters=3)
kmeans.fit(ne_30_100_embedds_2d)
labels = kmeans.labels_

# Create a DataFrame for Plotly
df = pd.DataFrame({'x': embedding_vectors_2d[:, 0],
                   'y': embedding_vectors_2d[:, 1],
                   'cluster': labels,
                   'text': text})

# Create a scatter plot with hover text
fig = px.scatter(df, x='x', y='y', color='cluster', hover_data=['text'])

# Customize the plot (add labels, titles, etc. if needed)
fig.update_layout(
    xaxis_title="PCA Component 1",
    yaxis_title="PCA Component 2",
    title="K-Means Clustering of Text Data (PCA Visualization)"
)

# Show the interactive plot
fig.show()









NameError: name 'get_texts_from_offsets' is not defined

In [ ]:
#make sure you are sending it in the right way for the method

In [19]:
normalized_2ne = normalize(ne_30_50_embedds, axis=1)
normalized_isa = normalize(isa_11_50_embedds, axis=1)

cos_sim = cosine_similarity(normalized_2ne, normalized_isa)

svd = TruncatedSVD(n_components=2)

svd_result = svd.fit_transform(cos_sim)

texts1 = get_texts_from_offset(ne_30_50_offsets, '2-ne/30')
texts2 = get_texts_from_offset(isa_11_50_offsets, 'isa/11')

plot_truncatedSVD(cos_sim, texts1, texts2)

'c:\\Users\\rylee\\OneDrive\\BYU\\CS_301R\\gospel-connections\\data'